In [ ]:
#ALIYA KAMAL, BU# 50548366

In [44]:
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score, accuracy_score

## Part 1: Data exploration and preprocessing

In [45]:
#Complete the following tasks within your code:
#1. [2 points] Explore the data following whatever techniques you wish (just be sure you include enough
#exploratory analysis so that you’re able to answer the questions for the write-up).
#load data 
#UB represents universal bank
ub = pd.read_csv('UniversalBank_unprocessed.csv')
ub.head()


,ID,Age,Experience,Income,Family,CCAvg,Education,Mortgage,CD Account,Online,CreditCard,Personal Loan
0,1,39,13.0,58.0,3,2.10,Undergraduate,169,0,1,0,0
1,2,51,25.0,18.0,1,0.30,Advanced,93,0,0,1,0
2,3,43,13.0,38.0,3,2.00,Advanced,0,0,1,0,0
3,4,37,12.0,60.0,4,2.10,Advanced,217,0,1,0,0
4,5,23,NaN,149.0,1,6.33,Undergraduate,305,0,0,1,0


In [46]:
#exploring data
ub.shape

(1117, 12)

In [47]:
#If dataset is balanced?
ub['Personal Loan'].value_counts()

Personal Loan
0    637
1    480
Name: count, dtype: int64

In [48]:
#check for Null values
ub.isnull().sum()

ID               0
Age              0
Experience       4
Income           3
Family           0
CCAvg            0
Education        0
Mortgage         0
CD Account       0
Online           0
CreditCard       0
Personal Loan    0
dtype: int64

In [49]:
ub.describe()

,ID,Age,Experience,Income,Family,CCAvg,Mortgage,CD Account,Online,CreditCard,Personal Loan
count,1117.000000,1117.000000,1113.000000,1114.000000,1117.000000,1117.000000,1117.000000,1117.000000,1117.000000,1117.000000,1117.000000
mean,559.000000,45.456580,20.263252,99.535009,2.502238,2.695801,72.025067,0.148612,0.601611,0.293644,0.429722
std,322.594431,11.514723,11.514062,54.566059,1.147400,2.114854,127.799025,0.355865,0.489786,0.455635,0.495258
min,1.000000,23.000000,-2.000000,8.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,280.000000,35.000000,10.000000,50.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,559.000000,45.000000,20.000000,95.000000,3.000000,2.200000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,838.000000,55.000000,30.000000,145.000000,4.000000,4.000000,114.000000,0.000000,1.000000,1.000000,1.000000
max,1117.000000,67.000000,42.000000,218.000000,4.000000,10.000000,617.000000,1.000000,1.000000,1.000000,1.000000


In [50]:
#Data type
ub.dtypes

ID                 int64
Age                int64
Experience       float64
Income           float64
Family             int64
CCAvg            float64
Education         object
Mortgage           int64
CD Account         int64
Online             int64
CreditCard         int64
Personal Loan      int64
dtype: object

In [51]:
#2. [1 point] Drop predictors as you see necessary.
#will drop age? education?
ub1 = ub.drop(["ID"], axis = 1)


In [52]:
#3. [1 point] Deal with NA values using any approach you wish.
#Experience and Income, both are numerical 
#either drop NA or get mean? I won't see an issue using mean but drop 4+3=7 out of 1172 will not have large impact either, but as bank's target is to 
#send personal offers to target customers I won't drop NA's.
# let's fill the NA values in "Experience" and "Income" with the column mean

ub_imp = ub1.fillna(value = {"Experience": ub["Experience"].mean(), "Income": ub["Income"].mean()}).copy()
ub_imp.isnull().sum()

Age              0
Experience       0
Income           0
Family           0
CCAvg            0
Education        0
Mortgage         0
CD Account       0
Online           0
CreditCard       0
Personal Loan    0
dtype: int64

In [53]:
#4. [1 point] Deal with categorical variables how you wish.
#Education column is categorical, count values
ub_imp['Education'].value_counts()

Education
Undergraduate    389
Advanced         383
Masters          345
Name: count, dtype: int64

In [54]:
#lets deal categorical data
ub_imp["Education"] = ub_imp["Education"].astype("category")
ub_imp.dtypes

Age                 int64
Experience        float64
Income            float64
Family              int64
CCAvg             float64
Education        category
Mortgage            int64
CD Account          int64
Online              int64
CreditCard          int64
Personal Loan       int64
dtype: object

In [55]:
#Flagging categorical data
ub_imp= pd.get_dummies(ub_imp,dtype=int)
ub_imp.head()

,Age,Experience,Income,Family,CCAvg,Mortgage,CD Account,Online,CreditCard,Personal Loan,Education_Advanced,Education_Masters,Education_Undergraduate
0,39,13.000000,58.0,3,2.10,169,0,1,0,0,0,0,1
1,51,25.000000,18.0,1,0.30,93,0,0,1,0,1,0,0
2,43,13.000000,38.0,3,2.00,0,0,1,0,0,1,0,0
3,37,12.000000,60.0,4,2.10,217,0,1,0,0,1,0,0
4,23,20.263252,149.0,1,6.33,305,0,0,1,0,0,0,1


In [56]:
#Check corr
ub_imp.corr()

,Age,Experience,Income,Family,CCAvg,Mortgage,CD Account,Online,CreditCard,Personal Loan,Education_Advanced,Education_Masters,Education_Undergraduate
Age,1.000000,0.991656,-0.043417,-0.046942,-0.036068,0.020851,0.024537,0.034029,0.024464,-0.029408,0.034434,-0.023320,-0.011692
Experience,0.991656,1.000000,-0.039570,-0.053761,-0.038674,0.026415,0.025698,0.028432,0.032929,-0.031698,0.014191,-0.025563,0.010652
Income,-0.043417,-0.039570,1.000000,-0.045924,0.628746,0.250918,0.257754,0.016805,-0.013512,0.720526,0.044394,0.018037,-0.061724
Family,-0.046942,-0.053761,-0.045924,1.000000,-0.012103,0.029502,0.025519,0.024708,0.012454,0.083456,-0.043344,0.065431,-0.020269
CCAvg,-0.036068,-0.038674,0.628746,-0.012103,1.000000,0.131017,0.197364,0.011636,0.003410,0.496695,0.028772,0.010219,-0.038577
Mortgage,0.020851,0.026415,0.250918,0.029502,0.131017,1.000000,0.142190,0.004240,0.037236,0.195850,0.017413,-0.024447,0.006359
CD Account,0.024537,0.025698,0.257754,0.025519,0.197364,0.142190,1.000000,0.262870,0.377198,0.349109,0.026943,0.063891,-0.088806
Online,0.034029,0.028432,0.016805,0.024708,0.011636,0.004240,0.262870,1.000000,0.002696,0.008225,-0.043985,0.092790,-0.046164
CreditCard,0.024464,0.032929,-0.013512,0.012454,0.003410,0.037236,0.377198,0.002696,1.000000,0.008144,0.027061,0.028476,-0.054577
Personal Loan,-0.029408,-0.031698,0.720526,0.083456,0.496695,0.195850,0.349109,0.008225,0.008144,1.000000,0.153983,0.132088,-0.281517


In [61]:
#Experience and Age are highly correlated, I gave it a thought but wanted to check Corr first before dropping
#As experience had 4 Null values filled with mean but, Age had no Null so I'll drop Experience at this point
ub_imp1=ub_imp.drop(['Experience'],axis=1)

In [62]:
#Choose predictors and response variables first
predictor_df = ub_imp1[['Age','Income','Family','CCAvg', 'Mortgage','CD Account', 'Online','CreditCard','Education_Advanced','Education_Masters','Education_Undergraduate']]
response_df = ub_imp1['Personal Loan']

In [63]:
#5. [1 point] Scale the predictors how you wish S_Scaler is Zscore scalar aka standard scalar
z_score_norm=preprocessing.StandardScaler()
predictor_df_normalize=z_score_norm.fit_transform(predictor_df)
predictor_df_normalize= pd.DataFrame(predictor_df_normalize,columns= predictor_df.columns)
predictor_df_normalize.head(10)

,Age,Income,Family,CCAvg,Mortgage,CD Account,Online,CreditCard,Education_Advanced,Education_Masters,Education_Undergraduate
0,-0.560975,-0.762554,0.434011,-0.281848,0.759148,-0.417796,0.813758,-0.644760,-0.722356,-0.668499,1.368015
1,0.481636,-1.496927,-1.309840,-1.133352,0.164198,-0.417796,-1.228866,1.550964,1.384359,-0.668499,-0.730986
2,-0.213438,-1.129740,0.434011,-0.329154,-0.563833,-0.417796,0.813758,-0.644760,1.384359,-0.668499,-0.730986
3,-0.734744,-0.725836,1.305937,-0.281848,1.134906,-0.417796,0.813758,-0.644760,1.384359,-0.668499,-0.730986
4,-1.951123,0.908143,-1.309840,1.719186,1.823796,-0.417796,-1.228866,1.550964,-0.722356,-0.668499,1.368015
5,1.263594,-1.276615,1.305937,-0.660294,0.359905,-0.417796,0.813758,1.550964,-0.722356,-0.668499,1.368015
6,1.002941,-1.258256,0.434011,-1.227964,-0.563833,-0.417796,0.813758,-0.644760,-0.722356,1.495888,-0.730986
7,-0.039670,0.412442,1.305937,0.664267,-0.563833,-0.417796,0.813758,-0.644760,-0.722356,1.495888,-0.730986
8,0.916057,-1.056303,1.305937,-0.092625,-0.563833,-0.417796,-1.228866,-0.644760,-0.722356,1.495888,-0.730986
9,0.394752,0.228849,-1.309840,-0.991435,-0.563833,-0.417796,-1.228866,-0.644760,-0.722356,-0.668499,1.368015


In [64]:
predictor_df_normalize.describe()

,Age,Income,Family,CCAvg,Mortgage,CD Account,Online,CreditCard,Education_Advanced,Education_Masters,Education_Undergraduate
count,1.117000e+03,1.117000e+03,1.117000e+03,1.117000e+03,1.117000e+03,1117.000000,1.117000e+03,1.117000e+03,1.117000e+03,1.117000e+03,1.117000e+03
mean,1.172841e-16,-1.526681e-16,9.859814e-17,7.633405e-17,3.816702e-17,0.000000,-5.566024e-17,6.917773e-17,-7.633405e-17,5.088936e-17,3.816702e-17
std,1.000448e+00,1.000448e+00,1.000448e+00,1.000448e+00,1.000448e+00,1.000448,1.000448e+00,1.000448e+00,1.000448e+00,1.000448e+00,1.000448e+00
min,-1.951123e+00,-1.680520e+00,-1.309840e+00,-1.275269e+00,-5.638332e-01,-0.417796,-1.228866e+00,-6.447605e-01,-7.223561e-01,-6.684992e-01,-7.309861e-01
25%,-9.085120e-01,-9.094287e-01,-1.309840e+00,-8.022118e-01,-5.638332e-01,-0.417796,-1.228866e+00,-6.447605e-01,-7.223561e-01,-6.684992e-01,-7.309861e-01
50%,-3.966962e-02,-2.818171e-02,4.340114e-01,-2.345426e-01,-5.638332e-01,-0.417796,8.137582e-01,-6.447605e-01,-7.223561e-01,-6.684992e-01,-7.309861e-01
75%,8.291728e-01,8.347060e-01,1.305937e+00,6.169612e-01,3.285920e-01,-0.417796,8.137582e-01,1.550964e+00,1.384359e+00,1.495888e+00,1.368015e+00
max,1.871784e+00,2.174936e+00,1.305937e+00,3.455307e+00,4.266222e+00,2.393515,8.137582e-01,1.550964e+00,1.384359e+00,1.495888e+00,1.368015e+00


## Part 2: $k$-NN

In [65]:
# partition data into train and test sets

X_classifier = predictor_df_normalize
y_classifier = response_df
train_X_classifier, test_X_classifier, train_y_classifier, test_y_classifier = train_test_split(X_classifier, 
                                                            y_classifier, test_size=0.3, random_state=616, 
                                                                                    stratify = y_classifier)

In [66]:
#Complete the following tasks within your code:
#1. [5 points] Train and test a k-NN model with k=5

knn = KNeighborsClassifier(n_neighbors=5).fit(train_X_classifier, train_y_classifier)
predicted_y_training = knn.predict(train_X_classifier)
f1_score(train_y_classifier, predicted_y_training)


0.9408194233687405

In [67]:
# performance of k-NN on test data
predicted_y_test = knn.predict(test_X_classifier)
f1_score(test_y_classifier, predicted_y_test)

0.8771929824561403

In [79]:
#2. [4 points] Find the optimal value of k
results = []
for k in range(1, 40):
    knn2 = KNeighborsClassifier(n_neighbors=k).fit(train_X_classifier, train_y_classifier)
    results.append({
        'k': k,
        'f1_score_train': f1_score(train_y_classifier, knn2.predict(train_X_classifier)),
        'f1_score_test': f1_score(test_y_classifier, knn2.predict(test_X_classifier))
    })

# Convert results to a pandas data frame
results = pd.DataFrame(results)
print(results)

     k  f1_score_train  f1_score_test
0    1        1.000000       0.881119
1    2        0.936709       0.844961
2    3        0.957704       0.889680
3    4        0.940994       0.856061
4    5        0.940819       0.877193
5    6        0.926677       0.892193
6    7        0.927803       0.895833
7    8        0.915094       0.889706
8    9        0.919084       0.900709
9   10        0.901099       0.888060
10  11        0.909091       0.909747
11  12        0.897638       0.884758
12  13        0.901235       0.898551
13  14        0.889590       0.856061
14  15        0.904025       0.882353
15  16        0.887480       0.868914
16  17        0.901716       0.901818
17  18        0.887122       0.881481
18  19        0.893417       0.893773
19  20        0.876800       0.863636
20  21        0.892744       0.880597
21  22        0.870968       0.862595
22  23        0.884311       0.877323
23  24        0.874598       0.870229
24  25        0.882166       0.877323
25  26      

## Part 3: Logistic regression and model comparison

In [ ]:
#Complete the following tasks within your code:
#1. [5 points] Train and test a logistic regression model

In [69]:
# partition data into train and test sets

X = predictor_df_normalize
y = response_df
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3, random_state=616)

In [70]:
# train the LR model

logistic_model = LogisticRegression(solver='lbfgs').fit(train_X, train_y)

In [71]:
# print the coefficients

print('intercept ', logistic_model.intercept_)
print(pd.DataFrame({'coeff': logistic_model.coef_[0]}, index=X.columns))

intercept  [-0.67329418]
                            coeff
Age                      0.006730
Income                   2.673612
Family                   0.533362
CCAvg                    0.429365
Mortgage                 0.208000
CD Account               1.243872
Online                  -0.337532
CreditCard              -0.473276
Education_Advanced       0.564737
Education_Masters        0.478093
Education_Undergraduate -1.026319


In [72]:
# print performance metric (i.e accuracy) on training set

predicted_y_training = logistic_model.predict(train_X)
print(accuracy_score(train_y, predicted_y_training))

0.9014084507042254


In [74]:
# what about the f_1 score?

f1_score(train_y, predicted_y_training)

0.8842105263157896

In [75]:
#extra step (not asked though)
# changing the cutoff threshold
THRESHOLD = 0.5
predicted_y_training = np.where(logistic_model.predict_proba(train_X)[:,1] > THRESHOLD, 1, 0)

pd.DataFrame(data=[accuracy_score(train_y, predicted_y_training), f1_score(train_y, predicted_y_training)], 
             index=["accuracy", "f1"])

,0
accuracy,0.901408
f1,0.884211


In [76]:
# how did our model perform in prediction?

print(accuracy_score(test_y, predicted_y_test))

0.5386904761904762


In [77]:
# what about the f_1 score?

f1_score(test_y, predicted_y_test)

0.4599303135888501